In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, LeaveOneOut, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc , classification_report, confusion_matrix
from ipywidgets import interact, interactive, widgets, interact_manual, Dropdown

from catboost import CatBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from sklearn.svm import SVC

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import re

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'catboost'

In [ ]:
data = pd.read_csv('/kaggle/input/handwriting-data-to-detect-alzheimers-disease/data.csv')

print("Shape of the data:", data.shape)
data.head()